In [1]:
!pip install bokeh

In [2]:
from ipywidgets import interact, widgets
import numpy as np
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import row, column, layout
from IPython.display import display
from bokeh.models import ColumnDataSource, Arrow, OpenHead
import warnings
warnings.filterwarnings('ignore')
output_notebook()

Loading BokehJS ...

In [3]:
t = np.linspace(0, 100, 10001)
i = np.zeros((len(t)))
u_t = np.sin(t)

u_plot = figure(plot_width=500, plot_height=500) #y_range=(-80, 100)
phase_plane_plot = figure(plot_width=500, plot_height=500)

u_shape = u_plot.line(t, u_t, line_width=3, line_alpha=0.6)
i_shape = u_plot.line(t, i,line_width=3, line_alpha=0.6, line_color='red')
phase_plane_F = phase_plane_plot.line(np.array([0]), np.array([0]), line_width=3, line_alpha=0.6)
phase_plane_G = phase_plane_plot.line(np.array([0]), np.array([0]), line_width=3, line_alpha=0.6)
phase_plane_point = phase_plane_plot.circle(np.array([0]), np.array([0]), fill_color="white", size=8)
phase_plane_threshold = phase_plane_plot.ray(x=[0, 0], y=[0, 0], length=0, angle=[90, 270], angle_units="deg", line_width=2, line_dash='dashed')


cds = ColumnDataSource(data=dict(x_start=[], y_start=[], x_end=[], y_end=[]))
phase_plane_vec = Arrow(end=OpenHead(size = 5), source=cds, line_color='red', line_width=1, x_start='x_start', y_start='y_start', x_end='x_end', y_end='y_end')
phase_plane_plot.add_layout(phase_plane_vec)


layout = row(u_plot, phase_plane_plot)

In [4]:
def build_I_t(input_time_0, input_value_0, input_time_1, input_value_1, input_time_2, input_value_2, input_time_3, input_value_3, input_time_4, input_value_4, input_type, input_constant, input_button_count):
    i = np.zeros(len(t))
    input_time = [input_time_0, input_time_1,input_time_2 ,input_time_3, input_time_4]
    input_value = [input_value_0, input_value_1,input_value_2 ,input_value_3, input_value_4]
    if input_type == 'Pulse':
        for j in range(len(t)):
            for k in range(input_button_count):
                if (t[j] >= input_time[k][0]) and (t[j] <= input_time[k][1]):
                    i[j] += input_value[k]
    elif input_type == 'Ramp':
        count = [0,0,0,0,0]
        for j in range(len(t)):
            for k in range(input_button_count):
                if (t[j] >= input_time[k][0]) and (t[j] <= input_time[k][1]):
                    i[j] += 0.01*count[k]
                    count[k] += 1
    elif input_type == 'Constant':
        i = np.full(len(t), input_constant)
    i_shape.data_source.data['y'] = i
    return i

In [5]:
leaky_max_potential = 80
leaky_u_linespace = np.linspace(-100, 100, 51)
def build_leaky(input_type, i, u_rest, time_constant, resistance, threshold, reset):
    
    leaky_u = np.full((len(t)), u_rest)
    
    d_leaky_u = (resistance*i[0])/time_constant
    thershold_flag = False
    for j in range(1, len(i)):
        if thershold_flag:
            leaky_u[j] = reset
            d_leaky_u = (resistance*i[j] + u_rest - leaky_u[j])/time_constant
            thershold_flag = False
            continue
        leaky_u[j] = leaky_u[j-1] + (0.01 * d_leaky_u)
        if leaky_u[j] > threshold:
            leaky_u[j] = leaky_max_potential
            thershold_flag = True
        d_leaky_u = (resistance*i[j] + u_rest - leaky_u[j])/time_constant
    u_shape.data_source.data['y'] = leaky_u
    
    phase_plane_F.data_source.data['x'] = leaky_u_linespace
    x_start = leaky_u_linespace
    y_start = np.zeros((len(x_start)))
    y_end = np.zeros((len(x_start)))
    if input_type == 'Constant':
        phase_plane_F.data_source.data['y'] = (resistance*i[0] + u_rest - leaky_u_linespace)/time_constant
        x_end = x_start + ((resistance*i[0] + u_rest - x_start)/time_constant) * 0.01
    else:
        phase_plane_F.data_source.data['y'] = (u_rest - x_start)/time_constant
        x_end = x_start + ((u_rest - x_start)/time_constant) * 0.01
    phase_plane_vec.source.data = {'x_start':x_start, 'y_start':y_start, 'x_end':x_end, 'y_end':y_end}
    phase_plane_vec.source.stream({'x_start':[], 'y_start':[], 'x_end':[], 'y_end':[]})
    phase_plane_threshold.data_source.data['x'] = [threshold, threshold]
    

In [6]:
FN_u_linespace = np.linspace(-5, 5, 1001)
def build_FN(input_type, i, epsilon, resistance, b0, b1, start_U, start_W):
    FN_u, FN_w = np.zeros((len(t))), np.zeros((len(t)))
    FN_u[0] = start_U
    FN_w[0] = start_W
    for j in range(1, len(t)):
        d_u = ((FN_u[j-1] -(np.power(FN_u[j-1], 3)/3) - FN_w[j-1])+ resistance*i[j-1])
        d_w = (b0 + b1*FN_u[j-1] - FN_w[j-1])*epsilon
        FN_u[j] = FN_u[j-1] + 0.01 * d_u
        FN_w[j] = FN_w[j-1] + 0.01 * d_w
   
    u_shape.data_source.data['y'] = FN_u
    phase_plane_point.data_source.data['x'] = np.array([start_U])
    phase_plane_point.data_source.data['y'] = np.array([start_W])
    phase_plane_F.data_source.data['x'] = FN_u_linespace
    phase_plane_G.data_source.data['x'] = FN_u_linespace
    
    if input_type == 'Constant':
        phase_plane_F.data_source.data['y'] = FN_u_linespace - (np.power(FN_u_linespace, 3)/3) + resistance*i[0]
    else:
        phase_plane_F.data_source.data['y'] = FN_u_linespace - (np.power(FN_u_linespace, 3)/3)
    
    phase_plane_G.data_source.data['y'] = b0 + (b1 * FN_u_linespace)
    
    vec_u = np.linspace(FN_u_linespace[0], FN_u_linespace[-1], 21)
    vec_w = np.linspace(min(phase_plane_F.data_source.data['y']), max(phase_plane_F.data_source.data['y']), 21)
    x_start = np.zeros((len(vec_u)*len(vec_w)))
    y_start = np.zeros((len(vec_u)*len(vec_w)))
    x_end = np.zeros((len(vec_u)*len(vec_w)))
    y_end = np.zeros((len(vec_u)*len(vec_w)))
    for j in range(len(vec_u)):
        for k in range(len(vec_w)):
            index = j*len(vec_u)+k
            x_start[index] = vec_u[j]
            y_start[index] = vec_w[k]
            if input_type == 'Constant':
                x_end[index] = vec_u[j] + (((vec_u[j] -(np.power(vec_u[j], 3)/3) - vec_w[k]) + resistance*i[0]))*0.01
            else:
                x_end[index] = vec_u[j] + ((vec_u[j] -(np.power(vec_u[j], 3)/3) - vec_w[k]))*0.01 
            y_end[index] = vec_w[k] + ((b0 + b1*vec_u[j] - vec_w[k])*epsilon)*0.01
    phase_plane_vec.source.data = {'x_start':x_start, 'y_start':y_start, 'x_end':x_end, 'y_end':y_end}
    phase_plane_vec.source.stream({'x_start':[], 'y_start':[], 'x_end':[], 'y_end':[]})
    

In [7]:
def HH_alpha_n(Vm):
    return (0.01 * (10.0 - Vm)) / (np.exp(1.0 - (0.1 * Vm)) - 1.0)

def HH_beta_n(Vm):
    return 0.125 * np.exp(-Vm / 80.0)

# Sodium ion-channel rate functions

def HH_alpha_m(Vm):
    return (0.1 * (25.0 - Vm)) / (np.exp(2.5 - (0.1 * Vm)) - 1.0)

def HH_beta_m(Vm):
    return 4.0 * np.exp(-Vm / 18.0)

def HH_alpha_h(Vm):
    return 0.07 * np.exp(-Vm / 20.0)

def HH_beta_h(Vm):
    return 1.0 / (np.exp(3.0 - (0.1 * Vm)) + 1.0)

def HH_n0(u=0.0):
    return HH_alpha_n(u) / (HH_alpha_n(u) + HH_beta_n(u))

def HH_m0(u=0.0):
    return HH_alpha_m(u) / (HH_alpha_m(u) + HH_beta_m(u))

def HH_h0(u=0.0):
    return HH_alpha_h(u) / (HH_alpha_h(u) + HH_beta_h(u))

def HH_Taw_n(u=0.0):
    return (1.0 / (HH_alpha_n(u) + HH_beta_n(u)))

def HH_Taw_m(u=0.0):
    return (1.0 / (HH_alpha_m(u) + HH_beta_m(u)))

def HH_Taw_h(u=0.0):
    return (1.0 / (HH_alpha_h(u) + HH_beta_h(u)))

def m(u , old_m = HH_m0()):
    return ((HH_alpha_m(u) * (1.0 - old_m)) - (HH_beta_m(u) * old_m))

def n(u, old_n = HH_n0()):
    return ((HH_alpha_n(u) * (1.0 - old_n)) - (HH_beta_n(u) * old_n))

def h(u , old_h = HH_h0()):
    return ((HH_alpha_h(u) * (1.0 - old_h)) - (HH_beta_h(u) * old_h))


def build_HH(input_type, i, input_gna, input_Ena, input_gk, input_Ek, input_gl, input_El, input_capacity, start_U):
    HH_u = np.zeros((len(t)))
    HH_u[0] = start_U
    m1 , n1 , h1 = HH_m0() , HH_n0() , HH_h0()
    for j in range(1, len(t)): 
        HH_GNA = (input_gna * np.power(m1,3)*h1*(HH_u[j-1]-input_Ena))
        HH_GK = (input_gk * np.power(n1,4)*(HH_u[j-1]-input_Ek))
        HH_GL = (input_gl * (HH_u[j-1]-input_El))
        HH_du = (1/(input_capacity))*(-HH_GNA - HH_GK - HH_GL + i[j-1] )
        HH_u[j] = HH_u[j-1] + 0.01 * HH_du
        m1 = m1 + 0.01*m(HH_u[j-1], m1)
        n1 = n1 + 0.01*n(HH_u[j-1], n1)
        h1 = h1 + 0.01*h(HH_u[j-1], h1)
    u_shape.data_source.data['y'] = HH_u

In [8]:
Gleaky_u_linespace = np.linspace(-100, 1, 51)
def build_GLeaky(input_type, i, u_rest, time_constant, resistance, threshold, reset, sharpness, v_rheobase):
    
    Gleaky_u = np.full((len(t)), u_rest)
    d_Gleaky_u = (sharpness*np.exp((Gleaky_u[0]-v_rheobase)/sharpness) + resistance*i[0])/time_constant
    thershold_flag = False
    
    for j in range(1, len(i)):
        if thershold_flag:
            Gleaky_u[j] = reset
            d_Gleaky_u = (sharpness*np.exp((Gleaky_u[j]-v_rheobase)/sharpness) + resistance*i[j] + u_rest - Gleaky_u[j])/time_constant
            thershold_flag = False
            continue
        Gleaky_u[j] = Gleaky_u[j-1] + (0.01 * d_Gleaky_u)
        if Gleaky_u[j] >= threshold:
            Gleaky_u[j] = threshold
            thershold_flag = True
        d_Gleaky_u = (sharpness*np.exp((Gleaky_u[j]-v_rheobase)/sharpness) + resistance*i[j] + u_rest - Gleaky_u[j])/time_constant
    u_shape.data_source.data['y'] = Gleaky_u
    
    phase_plane_F.data_source.data['x'] = Gleaky_u_linespace
    x_start = Gleaky_u_linespace
    y_start = np.zeros((len(x_start)))
    y_end = np.zeros((len(x_start)))
    if input_type == 'Constant':
        phase_plane_F.data_source.data['y'] = (sharpness*np.exp((x_start-v_rheobase)/sharpness) + resistance*i[0] + u_rest - x_start)/time_constant
        x_end = x_start + ((sharpness*np.exp((x_start-v_rheobase)/sharpness) + resistance*i[0] + u_rest - x_start)/time_constant) * 0.01
    else:
        phase_plane_F.data_source.data['y'] = (sharpness*np.exp((x_start-v_rheobase)/sharpness) + u_rest - x_start)/time_constant
        x_end = x_start + ((sharpness*np.exp((x_start-v_rheobase)/sharpness) + u_rest - x_start)/time_constant) * 0.01
    phase_plane_vec.source.data = {'x_start':x_start, 'y_start':y_start, 'x_end':x_end, 'y_end':y_end}
    phase_plane_vec.source.stream({'x_start':[], 'y_start':[], 'x_end':[], 'y_end':[]})
    phase_plane_threshold.data_source.data['x'] = [threshold, threshold]

In [9]:
ML_u_linespace = np.linspace(-80, 40, 201)
def ML_w0(u, u3, u4):
    return (1/2)*(1 + np.tanh((u - u3)/u4))
def ML_m0(u, u1, u2):
    return (1/2)*(1 + np.tanh((u - u1)/u2))
def ML_taw(u, u3, u4, taw_w):
    return taw_w/np.cosh((u - u3)/(2*u4))
def build_ML(input_type, i, input_gna, input_Ena, input_gk, input_Ek, 
             input_gl, input_El, input_capacity, start_U , start_w_hat, taw_w, u1, u2, u3, u4):
    ML_u, ML_w = np.zeros((len(t))), np.zeros((len(t)))
    ML_u[0] = start_U
    ML_w[0] = start_w_hat
    for j in range(1, len(t)):
        d_u = (1/input_capacity)*((i[j-1]) - ((input_gk) * ML_w[j-1] * (ML_u[j-1] - input_Ek)) - ((input_gna) * np.power(ML_m0(ML_u[j-1], u1, u2),3) * (ML_u[j-1] - input_Ena)) - (input_gl* (ML_u[j-1] - input_El)))
        d_w = ((-1)/ML_taw(ML_u[j-1], u3, u4, taw_w))*(ML_w[j-1] - ML_w0(ML_u[j-1], u3, u4))
        ML_u[j] = ML_u[j-1] + 0.01 * d_u
        ML_w[j] = ML_w[j-1] + 0.01 * d_w
    u_shape.data_source.data['y'] = ML_u
    phase_plane_point.data_source.data['x'] = np.array([start_U])
    phase_plane_point.data_source.data['y'] = np.array([start_w_hat])
    phase_plane_F.data_source.data['x'] = ML_u_linespace
    phase_plane_G.data_source.data['x'] = ML_u_linespace
    if input_type == 'Constant':
        phase_plane_F.data_source.data['y'] = (1/(input_gk*(ML_u_linespace-input_Ek))) * ((-1*input_gna*np.power(ML_m0(ML_u_linespace, u1, u2),3)*(ML_u_linespace-input_Ena))-(input_gl*(ML_u_linespace-input_El))+i[0])
    else:
        phase_plane_F.data_source.data['y'] = (1/(input_gk*(ML_u_linespace-input_Ek))) * ((-1*input_gna*np.power(ML_m0(ML_u_linespace, u1, u2),3)*(ML_u_linespace-input_Ena))-(input_gl*(ML_u_linespace-input_El)))  
    phase_plane_G.data_source.data['y'] = ML_w0(ML_u_linespace, u3, u4)
    
    vec_u = np.linspace(ML_u_linespace[0], ML_u_linespace[-1], 21)
    vec_w = np.linspace(min(phase_plane_F.data_source.data['y']), max(phase_plane_F.data_source.data['y']), 21)
    x_start = np.zeros((len(vec_u)*len(vec_w)))
    y_start = np.zeros((len(vec_u)*len(vec_w)))
    x_end = np.zeros((len(vec_u)*len(vec_w)))
    y_end = np.zeros((len(vec_u)*len(vec_w)))
    for j in range(len(vec_u)):
        for k in range(len(vec_w)):
            index = j*len(vec_u)+k
            x_start[index] = vec_u[j]
            y_start[index] = vec_w[k]
            if input_type == 'Constant':
                x_end[index] = vec_u[j] + 0.01*((i[0] / input_capacity) - ((input_gk / input_capacity) * vec_w[k] * (vec_u[j] - input_Ek)) - ((input_gna / input_capacity) * np.power(ML_m0(vec_u[j], u1, u2),3) * (vec_u[j] - input_Ena)) - (input_gl /input_capacity * (vec_u[j] - input_El)))
            else:
                x_end[index] = vec_u[j] - 0.01*(((input_gk / input_capacity) * vec_w[k] * (vec_u[j] - input_Ek)) - ((input_gna / input_capacity) * np.power(ML_m0(vec_u[j], u1, u2),3) * (vec_u[j] - input_Ena)) - (input_gl /input_capacity * (vec_u[j] - input_El)))
            y_end[index] = vec_w[k] + (((-1)/ML_taw(vec_u[j], u3, u4, taw_w))*(vec_w[k] - ML_w0(vec_u[j], u3, u4)))*0.01
    phase_plane_vec.source.data = {'x_start':x_start, 'y_start':y_start, 'x_end':x_end, 'y_end':y_end}
    phase_plane_vec.source.stream({'x_start':[], 'y_start':[], 'x_end':[], 'y_end':[]})
    



In [10]:
D1_u_linespace = np.linspace(-4, 4, 201)
def build_D1(input_type, i, input_b0, input_b1, input_eps, U_start, w_start):
    D1_u, D1_w = np.zeros((len(t))), np.zeros((len(t)))
    D1_u[0] = U_start
    D1_w[0] = w_start
    for j in range(1, len(t)):
        d_u = D1_u[j-1]-((1/3)*np.power(D1_u[j-1],3))-D1_w[j-1] + i[j-1]
        d_w = input_eps*(input_b0 + (input_b1 * D1_u[j-1])- D1_w[j-1])
        D1_u[j] = D1_u[j-1] + 0.01 * d_u
        D1_w[j] = D1_w[j-1] + 0.01 * d_w
    u_shape.data_source.data['y'] = D1_u
    phase_plane_point.data_source.data['x'] = np.array([U_start])
    phase_plane_point.data_source.data['y'] = np.array([w_start])
    phase_plane_F.data_source.data['x'] = D1_u_linespace
    phase_plane_G.data_source.data['x'] = D1_u_linespace
    if input_type == 'Constant':
        phase_plane_F.data_source.data['y'] = D1_u_linespace - ((1/3)*np.power(D1_u_linespace,3))+i[0]
    else:
        phase_plane_F.data_source.data['y'] =  D1_u_linespace - ((1/3)*np.power(D1_u_linespace,3)) 
    phase_plane_G.data_source.data['y'] = input_b1 * D1_u_linespace + input_b0
    
    vec_u = np.linspace(D1_u_linespace[0], D1_u_linespace[-1], 21)
    vec_w = np.linspace(min(phase_plane_F.data_source.data['y']), max(phase_plane_F.data_source.data['y']), 21)
    x_start = np.zeros((len(vec_u)*len(vec_w)))
    y_start = np.zeros((len(vec_u)*len(vec_w)))
    x_end = np.zeros((len(vec_u)*len(vec_w)))
    y_end = np.zeros((len(vec_u)*len(vec_w)))
    for j in range(len(vec_u)):
        for k in range(len(vec_w)):
            index = j*len(vec_u)+k
            x_start[index] = vec_u[j]
            y_start[index] = vec_w[k]
            if input_type == 'Constant':
                x_end[index] = vec_u[j] + 0.01*(vec_u[j]-((1/3)*np.power(vec_u[j],3))-vec_w[k] + i[j])
            else:
                x_end[index] = vec_u[j] - 0.01*(vec_u[j]-((1/3)*np.power(vec_u[j],3))-vec_w[k])
            y_end[index] = vec_w[k] + (input_eps*(input_b0 + (input_b1 * vec_u[j])- vec_w[k]))*0.01
    phase_plane_vec.source.data = {'x_start':x_start, 'y_start':y_start, 'x_end':x_end, 'y_end':y_end}
    phase_plane_vec.source.stream({'x_start':[], 'y_start':[], 'x_end':[], 'y_end':[]})

In [11]:
## Update
def update(model,input_time_0, input_value_0, input_time_1, input_value_1, input_time_2, input_value_2, input_time_3, input_value_3, input_time_4, input_value_4, input_type, input_constant,input_button_count, leaky_u_rest, leaky_time_constant, leaky_resistance, leaky_threshold, leaky_reset,
           FN_u_tc, FN_resistance, FN_b0, FN_b1, FN_start_U, FN_start_W, HH_input_gna, HH_input_Ena, HH_input_gk, HH_input_Ek, HH_input_gl, HH_input_El, HH_input_capacity, HH_start_U,
           ML_input_gna, ML_input_Ena, ML_input_gk, ML_input_Ek, ML_input_gl, ML_input_El, ML_input_capacity, ML_start_U, ML_start_w_hat, ML_taw_w, ML_u1, ML_u2, ML_u3, ML_u4, 
           GL_u_rest, GL_time_constant, GL_resistance, GL_threshold, GL_reset, GL_sharpness, GL_v_rheobase, D1_input_b0, D1_input_b1, D1_input_eps, D1_U_start, D1_w_start):
    
    # UI
    for k in range(5):
        if k < input_button_count:
            input_time_w[k].layout.display = 'flex'
            input_value_w[k].layout.display = 'flex'
        else:
            input_time_w[k].layout.display = 'none'
            input_value_w[k].layout.display = 'none'
    
    i = build_I_t(input_time_0, input_value_0, input_time_1, input_value_1, input_time_2, input_value_2, input_time_3, input_value_3, input_time_4, input_value_4, input_type, input_constant, input_button_count)
    if input_type == 'Pulse':
        input_pulse_ui.layout.display = 'block'
    else:
        input_pulse_ui.layout.display = 'none'
    if input_type == 'Ramp':
        input_ramp_ui.layout.display = 'block'
    else:
        input_ramp_ui.layout.display = 'none'
    if input_type == 'Constant':
        input_constant_ui.layout.display = 'block'
    else:
        input_constant_ui.layout.display = 'none'
    
    if model == 'Leaky':
        leaky_ui.layout.display = 'block'
        phase_plane_F.visible = True
        phase_plane_G.visible = False
        phase_plane_point.visible = False
        phase_plane_threshold.visible = True
        build_leaky(input_type, i, leaky_u_rest, leaky_time_constant, leaky_resistance, leaky_threshold, leaky_reset)
    else:
        leaky_ui.layout.display = 'none'

    
    if model == 'FN':
        FN_ui.layout.display = 'block'
        phase_plane_F.visible = True
        phase_plane_G.visible = True
        phase_plane_point.visible = True
        phase_plane_threshold.visible = False
        build_FN(input_type, i, FN_u_tc, FN_resistance, FN_b0, FN_b1, FN_start_U, FN_start_W)
    else:
        FN_ui.layout.display = 'none'
    
    if model == 'HH':
        HH_ui.layout.display = 'block'
        phase_plane_F.visible = False
        phase_plane_G.visible = False
        phase_plane_point.visible = False
        phase_plane_threshold.visible = False
        phase_plane_vec.source.data = {'x_start':[], 'y_start':[], 'x_end':[], 'y_end':[]}
        phase_plane_vec.source.stream({'x_start':[], 'y_start':[], 'x_end':[], 'y_end':[]})
        build_HH(input_type, i, HH_input_gna, HH_input_Ena, HH_input_gk, HH_input_Ek, HH_input_gl, HH_input_El, HH_input_capacity, HH_start_U)
    else:
        HH_ui.layout.display = 'none'
    
    if model == 'ML':
        ML_ui.layout.display = 'block'
        phase_plane_F.visible = True
        phase_plane_G.visible = True
        phase_plane_point.visible = True
        phase_plane_threshold.visible = False
        build_ML(input_type, i, ML_input_gna, ML_input_Ena, ML_input_gk, ML_input_Ek, ML_input_gl, ML_input_El, ML_input_capacity, 
                 ML_start_U, ML_start_w_hat, ML_taw_w, ML_u1, ML_u2, ML_u3, ML_u4)
    else:
        ML_ui.layout.display = 'none'
        
    if model == 'GLeaky':
        GL_ui.layout.display = 'block'
        phase_plane_F.visible = True
        phase_plane_G.visible = False
        phase_plane_point.visible = False
        phase_plane_threshold.visible = True
        build_GLeaky(input_type, i, GL_u_rest, GL_time_constant, GL_resistance, GL_threshold, GL_reset, GL_sharpness, GL_v_rheobase)
    else:
        GL_ui.layout.display = 'none'
    
    if model == 'D1':
        D1_ui.layout.display = 'block'
        phase_plane_F.visible = True
        phase_plane_G.visible = True
        phase_plane_point.visible = True
        phase_plane_threshold.visible = False
        build_D1(input_type, i, D1_input_b0, D1_input_b1, D1_input_eps, D1_U_start, D1_w_start)
    else:
        D1_ui.layout.display = 'none'    
    push_notebook()


In [12]:
## Widgets
input_type_w = widgets.ToggleButtons(options=['Pulse', 'Ramp', 'Constant'], description='Input Type:')

input_constant_w= widgets.FloatSlider(value=2,min=-0, max=100, step=0.1, description='Current Value:', style={'description_width': 'initial', }, readout_format='.1f',orientation='horizontal')
input_button_count_w = widgets.IntSlider(value=1,min=1, max=5, step=1, description='Signal Input Number:', style={'description_width': 'initial', }, readout_format='.1f',orientation='horizontal')


input_time_w, input_value_w = [], []
for k in range(5):
    input_time_w.append(widgets.FloatRangeSlider(value=[0, 10], min=0, max=100.0, step=0.01, description='Input Time '+str(k+1)+' (ms):',  style={'description_width': 'initial', },readout_format='.2f'))
    input_value_w.append(widgets.FloatSlider(value=1, min=-100, max=100.0, step=0.1, description='Current Value '+str(k+1)+' :', style={'description_width': 'initial', } ,readout_format='.1f'))
    if k > 0:
        input_time_w[k].layout.display = 'none'
        input_value_w[k].layout.display = 'none'

model_w = widgets.Dropdown(options=['HH', 'Leaky', 'FN', 'ML', 'GLeaky', 'D1'], value='Leaky', description='Model:')
## Leaky
leaky_u_rest_w = widgets.FloatSlider(value=-67,min=-120, max=0, step=0.1, description='Resting Potential(mV):', style={'description_width': 'initial', }, readout_format='.1f')
leaky_time_constant_w = widgets.FloatSlider(value=1.0, min=0.1, max=100.0, step=0.1, description='Time Constant (mS):', style={'description_width': 'initial', }, readout_format='.1f')
leaky_resistance_w =  widgets.FloatSlider(value=20.0, min=0.1, max=100.0, step=0.1, description='Resistance(K Ohm):', style={'description_width': 'initial', }, readout_format='.1f')
leaky_threshold_w = widgets.FloatSlider(value=30, min=-0, max=100, step=0.1, description='Threshold(mV):', style={'description_width': 'initial', }, readout_format='.1f')
leaky_reset_w = widgets.FloatSlider(value=-70, min=-150, max=0, step=0.1, description='Reseting Potential(mV):', style={'description_width': 'initial', }, readout_format='.1f')

##FN
FN_u_tc_w = widgets.FloatSlider(value=0.05, min=0., max=10, step=0.01, description='epsilon:', style={'description_width': 'initial', }, readout_format='.1f')
FN_b0_w = widgets.FloatSlider(value=2, min=-100, max=100.0, step=0.1, description='b0:', style={'description_width': 'initial', }, readout_format='.1f')
FN_b1_w = widgets.FloatSlider(value=1.5, min=-100, max=100.0, step=0.1, description='b1:', style={'description_width': 'initial', }, readout_format='.1f')
FN_resistance_w =  widgets.FloatSlider(value=1, min=0.1, max=100.0, step=0.1, description='Resistance:', style={'description_width': 'initial', }, readout_format='.1f')
FN_start_U_w =  widgets.FloatSlider(value=-2, min=-20, max=20.0, step=0.01, description='Start Point U:', style={'description_width': 'initial', }, readout_format='.2f')
FN_start_W_w =  widgets.FloatSlider(value=-.5, min=-20, max=20.0, step=0.01, description='Start Point W:', style={'description_width': 'initial', }, readout_format='.2f')

## HH
HH_input_gna_w = widgets.FloatSlider(value=120, min=0.1, max=200.0, step=0.1, description='g_na (nS/cm^2):', style={'description_width': 'initial', }, readout_format='.1f')
HH_input_Ena_w = widgets.FloatSlider(value=115, min=0.1, max=200.0, step=0.1, description='E_na (mV)):', style={'description_width': 'initial', }, readout_format='.1f')
HH_input_gk_w = widgets.FloatSlider(value=36, min=0.1, max=100.0, step=0.1, description='g_k (nS/cm^2):', style={'description_width': 'initial', }, readout_format='.1f')
HH_input_Ek_w = widgets.FloatSlider(value=-12, min=-100, max=100.0, step=0.1, description='E_k (mV):', style={'description_width': 'initial', }, readout_format='.1f')
HH_input_gl_w = widgets.FloatSlider(value=0.3, min=0.1, max=100.0, step=0.1, description='g_l(nS/cm^2):', style={'description_width': 'initial', }, readout_format='.1f')
HH_input_El_w = widgets.FloatSlider(value=10.6, min=-100, max=100.0, step=0.1, description='E_l (mV):', style={'description_width': 'initial', }, readout_format='.1f')
HH_input_capacity_w = widgets.FloatSlider(value=1, min=0.1, max=100.0, step=0.1, description='Capacity (microF/cm^2):', style={'description_width': 'initial', }, readout_format='.1f')
HH_start_U_w = widgets.FloatSlider(value=0, min=-100, max=100, step=0.1, description='Start point U:', style={'description_width': 'initial', }, readout_format='.1f')


## ML
ML_input_gna_w = widgets.FloatSlider(value=5, min=0.1, max=200.0, step=0.1, description='g_na (nS/cm^2):', style={'description_width': 'initial', }, readout_format='.1f')
ML_input_Ena_w =  widgets.FloatSlider(value=90, min=0.1, max=200.0, step=0.1, description='E_na (mV)):', style={'description_width': 'initial', }, readout_format='.1f')
ML_input_gk_w = widgets.FloatSlider(value=6, min=0.1, max=100.0, step=0.1, description='g_k (nS/cm^2):', style={'description_width': 'initial', }, readout_format='.1f')
ML_input_Ek_w = widgets.FloatSlider(value=-83, min=-100, max=100.0, step=0.1, description='E_k (mV):', style={'description_width': 'initial', }, readout_format='.1f')
ML_input_gl_w = widgets.FloatSlider(value=0.5, min=0.1, max=100.0, step=0.1, description='g_l(nS/cm^2):', style={'description_width': 'initial', }, readout_format='.1f')
ML_input_El_w = widgets.FloatSlider(value=-50, min=-100, max=100.0, step=0.1, description='E_l (mV):', style={'description_width': 'initial', }, readout_format='.1f')
ML_input_capacity_w = widgets.FloatSlider(value=1, min=0.1, max=100.0, step=0.1, description='Capacity (microF/cm^2):', style={'description_width': 'initial', }, readout_format='.1f')
ML_start_U_w = widgets.FloatSlider(value=-67, min=-100, max=100, step=0.1, description='Start point U:', style={'description_width': 'initial', }, readout_format='.1f')
ML_start_w_hat_w = widgets.FloatSlider(value=0.1, min=-100, max=100, step=0.1, description='Start point W:', style={'description_width': 'initial', }, readout_format='.1f')
ML_taw_w_w = widgets.FloatSlider(value=30, min=0.1, max=100, step=0.1, description='Time Constant W (mS):', style={'description_width': 'initial', }, readout_format='.1f')
ML_u1_w = widgets.FloatSlider(value=-2.6, min=-100, max=100, step=0.1, description='u1:', style={'description_width': 'initial', }, readout_format='.1f')
ML_u2_w = widgets.FloatSlider(value=23.5, min=-100, max=100, step=0.1, description='u2:', style={'description_width': 'initial', }, readout_format='.1f')
ML_u3_w = widgets.FloatSlider(value=24.4, min=-100, max=100, step=0.1, description='u3:', style={'description_width': 'initial', }, readout_format='.1f')
ML_u4_w = widgets.FloatSlider(value=23.2, min=-100, max=100, step=0.1, description='u4:', style={'description_width': 'initial', }, readout_format='.1f')
   

## G Leaky
GL_u_rest_w = widgets.FloatSlider(value=-67,min=-120, max=0, step=0.1, description='Resting Potentia l(mV):', style={'description_width': 'initial', }, readout_format='.1f')
GL_time_constant_w = widgets.FloatSlider(value=1.0, min=0.1, max=100.0, step=0.1, description='Time Constant:', style={'description_width': 'initial', }, readout_format='.1f')
GL_resistance_w = widgets.FloatSlider(value=20.0, min=0.1, max=100.0, step=0.1, description='Resistance:', style={'description_width': 'initial', }, readout_format='.1f')
GL_threshold_w = widgets.FloatSlider(value=40, min=-150, max=150, step=0.1, description='Threshold (mV):', style={'description_width': 'initial', }, readout_format='.1f')
GL_sharpness_w = widgets.FloatSlider(value=0.5, min=0, max=20, step=0.01, description='Sharpness (mV):', style={'description_width': 'initial', }, readout_format='.2f')
GL_reset_w = widgets.FloatSlider(value=-70, min=-150, max=0, step=0.1, description='Reseting Potential (mV):', style={'description_width': 'initial', }, readout_format='.1f')
GL_v_rheobase_w = widgets.FloatSlider(value=-3, min=-150, max=0, step=0.1, description='V rheobase (mV):', style={'description_width': 'initial', }, readout_format='.1f')

## D1
D1_input_b0_w = widgets.FloatSlider(value=0.9, min=-100, max=100.0, step=0.1, description='b0:', style={'description_width': 'initial', }, readout_format='.1f')
D1_input_b1_w = widgets.FloatSlider(value=1.0, min=-100, max=100.0, step=0.1, description='b1:', style={'description_width': 'initial', }, readout_format='.1f')
D1_input_eps_w = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.01, description='epsilon:', style={'description_width': 'initial', }, readout_format='.2f')
D1_U_start_w = widgets.FloatSlider(value=0.5, min=-100, max=100, step=0.1, description='Start point U:', style={'description_width': 'initial', }, readout_format='.1f')
D1_w_start_w = widgets.FloatSlider(value=-0.1, min=-20, max=20.0, step=0.1, description='Start point W:', style={'description_width': 'initial', }, readout_format='.1f')
    
# input UI
input_pulse_list = []
for k in range(5):
    input_pulse_list.append(input_time_w[k])
    input_pulse_list.append(input_value_w[k])


input_pulse_ui = widgets.VBox([input_button_count_w] + input_pulse_list )
input_ramp_ui =  widgets.VBox([input_button_count_w] +[x for x in input_time_w] )
input_constant_ui = widgets.VBox([input_constant_w])
input_ui = widgets.VBox([input_type_w ,input_pulse_ui, input_constant_ui, input_ramp_ui])
input_constant_ui.layout.display = 'none'
input_ramp_ui.layout.display = 'none'

# model UI
leaky_ui = widgets.VBox([leaky_u_rest_w, leaky_time_constant_w, leaky_resistance_w, leaky_threshold_w, leaky_reset_w])

FN_ui = widgets.VBox([FN_u_tc_w, FN_resistance_w, FN_b0_w, FN_b1_w, FN_start_U_w, FN_start_W_w])
FN_ui.layout.display = 'none'

HH_ui = widgets.VBox([HH_input_gna_w, HH_input_Ena_w, HH_input_gk_w, HH_input_Ek_w, HH_input_gl_w, HH_input_El_w, HH_input_capacity_w, HH_start_U_w])
HH_ui.layout.display = 'none'

ML_ui = widgets.VBox([ML_input_gna_w, ML_input_Ena_w, ML_input_gk_w, ML_input_Ek_w, ML_input_gl_w, ML_input_El_w, ML_input_capacity_w, ML_start_U_w,
                      ML_start_w_hat_w, ML_taw_w_w, ML_u1_w, ML_u2_w, ML_u3_w, ML_u4_w])
ML_ui.layout.display = 'none'

GL_ui = widgets.VBox([GL_u_rest_w, GL_time_constant_w, GL_resistance_w, GL_threshold_w, GL_reset_w, GL_sharpness_w, GL_v_rheobase_w])
GL_ui.layout.display = 'none'

D1_ui = widgets.VBox([D1_input_b0_w, D1_input_b1_w, D1_input_eps_w, D1_U_start_w, D1_w_start_w])
D1_ui.layout.display = 'none'

model_ui = widgets.VBox([model_w, leaky_ui, FN_ui, HH_ui, ML_ui, GL_ui, D1_ui])


all_widgets = widgets.interactive_output(update, {'model':model_w, 'input_time_0':input_time_w[0], 'input_value_0':input_value_w[0], 'input_time_1':input_time_w[1], 'input_value_1':input_value_w[1], 
        'input_time_2':input_time_w[2], 'input_value_2':input_value_w[2], 'input_time_3':input_time_w[3], 'input_value_3':input_value_w[3], 'input_time_4':input_time_w[4], 'input_value_4':input_value_w[4],
        'input_type':input_type_w, 'input_constant': input_constant_w,'input_button_count':input_button_count_w,
        'leaky_u_rest':leaky_u_rest_w, 'leaky_time_constant':leaky_time_constant_w, 'leaky_resistance':leaky_resistance_w, 'leaky_threshold':leaky_threshold_w, 
        'leaky_reset':leaky_reset_w, 'FN_u_tc':FN_u_tc_w, 'FN_resistance':FN_resistance_w, 'FN_b0':FN_b0_w, 'FN_b1':FN_b1_w, 
        'FN_start_U':FN_start_U_w, 'FN_start_W':FN_start_W_w, 'HH_input_gna': HH_input_gna_w, 'HH_input_Ena': HH_input_Ena_w, 'HH_input_gk': HH_input_gk_w,
        'HH_input_Ek': HH_input_Ek_w, 'HH_input_gl': HH_input_gl_w, 'HH_input_El': HH_input_El_w, 'HH_input_capacity': HH_input_capacity_w, 'HH_start_U': HH_start_U_w,
        'ML_input_gna': ML_input_gna_w, 'ML_input_Ena': ML_input_Ena_w, 'ML_input_gk': ML_input_gk_w,'ML_input_Ek': ML_input_Ek_w, 'ML_input_gl': ML_input_gl_w,
        'ML_input_El': ML_input_El_w, 'ML_input_capacity': ML_input_capacity_w, 'ML_start_U': ML_start_U_w, 'ML_start_w_hat':ML_start_w_hat_w,'ML_taw_w':ML_taw_w_w,
        'ML_u1':ML_u1_w, 'ML_u2':ML_u2_w, 'ML_u3':ML_u3_w, 'ML_u4':ML_u4_w, 'GL_u_rest':GL_u_rest_w, 'GL_time_constant':GL_time_constant_w, 'GL_resistance':GL_resistance_w,
        'GL_threshold':GL_threshold_w, 'GL_reset':GL_reset_w, 'GL_sharpness':GL_sharpness_w, 'GL_v_rheobase':GL_v_rheobase_w, 'D1_input_b0':D1_input_b0_w, 
        'D1_input_b1':D1_input_b1_w, 'D1_input_eps':D1_input_eps_w, 'D1_U_start':D1_U_start_w, 'D1_w_start':D1_w_start_w});




In [13]:
show(layout, notebook_handle=True);
display(widgets.HBox([input_ui, model_ui]), all_widgets);

Output()